In [5]:
# definitions for carrying out bleu score calculations for two outputs
import nltk
import subprocess
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

def bleu_score(hypothesis, reference):
    from nltk.translate.bleu_score import sentence_bleu
    return sentence_bleu([reference], hypothesis)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [2]:
# !pip install rouge-score
import rouge_score
from rouge_score import rouge_scorer

def calculate_rouge_score(hypothesis, reference):
   
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores['rougeL'].fmeasure



hypothesis = "The quick brown fox jumps over the lazy dog"
reference = "The quick brown fox jumped over the lazy dog"
rouge_l_fmeasure = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_l_fmeasure}")

hypothesis = "The slow purple fox jumps over the lazy fox"

rouge_Scores = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_Scores}")


ROUGE-L F-measure: 1.0
ROUGE-L F-measure: 0.6666666666666666


In [ ]:
# !pip install bert_score
subprocess.run(["pip", "install", "bert_score"]) 
import bert_score

def calculate_bertscore(hypotheses, references, model_type='bert-base-uncased'):
    P, R, F1 = bert_score.score(hypotheses, references, model_type=model_type, lang="en")

    return {
        'precision': P.mean().item(),
        'recall': R.mean().item(),
        'f1': F1.mean().item()
    }

# Example usage
hypotheses = ["The quick brown fox jumps over the lazy dog"]
references = ["The quick brown fox jumped over the lazy dog"]
bertscore_result = calculate_bertscore(hypotheses, references)
print(f"BERTScore: {bertscore_result}")

DEPRECATION: Loading egg at /Users/krishpatel/anaconda3/lib/python3.11/site-packages/litellm-1.40.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/krishpatel/anaconda3/lib/python3.11/site-packages/openai-0.27.7-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


/Users/krishpatel/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/krishpatel/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


: 

In [3]:
# ADD PERPLEXITY

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def calculate_perplexity(text, model_name='gpt2'):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    input_ids = tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        loss = model(input_ids, labels=input_ids)[0]
    return loss.item()

# Example usage
text = "The quick brown fox jumps over the lazy dog"
perplexity = calculate_perplexity(text)
print(f"Perplexity: {perplexity}")

Perplexity: 5.426182746887207


In [4]:
import openai
from openai import OpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key
client = OpenAI(api_key="sk-proj-4Brh6XVArOOG4c7DMUI7T3BlbkFJIZrEaupeqYNS23AIYpnz")

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# baseline example

def generate_persona_prompt(prompt, context):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will provide the personas only for user 2 \
     I will also provide the first few parts of the conversation between the 2 users with the \
     given persona, from which you will have to infer the person of user1 and respond accordingly. Given the persona of user 2 (your persona) and the beginning of the conversation, \
     you will need to reply to my prompt as if you were that user, and take on that user's personality \
     based on the description provided. Here are the personas of user 2, \
     and beginning of their conversation: {context}"},
    {"role": "user", "content": f"{prompt}"}
  ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
# persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
# text = "Tell me about your internship!"

# response = generate_persona_prompt(text, persona)
# print(f"Response: {response}")


In [ ]:
# finetuned example

def generate_persona_prompt_finetuned(prompt, context):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will provide the personas only for user 2 \
     I will also provide the first few parts of the conversation between the 2 users with the \
     given persona, from which you will have to infer the person of user1 and respond accordingly. Given the persona of user 2 (your persona) and the beginning of the conversation, \
     you will need to reply to my prompt as if you were that user, and take on that user's personality \
     based on the description provided. Here are the personas of user 2, \
     and beginning of their conversation: {context}"},
    {"role": "user", "content": f"{prompt}"}
  ]
    
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::9WTbkMz7",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
# persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
# text = "Tell me about your internship!"

# response = generate_persona_prompt_finetuned(text, persona)
# print(f"Response: {response}")


In [ ]:
import pandas as pd

def extract_conversation_parts(csv_filename):
    data = pd.read_csv(csv_filename)
    
    extracted_data = []
    
    for _, row in data.iterrows():
        user1_persona = row['user 1 personas']
        user2_persona = row['user 2 personas']
        conversation = row['Best Generated Conversation']
        
        conversation_lines = conversation.split('\n')
        
        if len(conversation_lines) >= 7:
            context = "\n".join(conversation_lines[:6])
            prompt = conversation_lines[6]
            full_continued_convo = "\n".join(conversation_lines[7:])
            single_continued_convo = conversation_lines[7]
            
            extracted_data.append({
                'user1_persona': user1_persona,
                'user2_persona': user2_persona,
                'context': context,
                'prompt': prompt,
                'finished_convo': full_continued_convo,
                'prompt_response': single_continued_convo
            })
        else:
            print(f"Warning: Conversation is too short in row {row.name}")

    return extracted_data

csv_filename = r'Synthetic-Persona-Chat_valid.csv'
extracted_data = extract_conversation_parts(csv_filename)

trunc_extracted_data = extracted_data[:15]

for data in trunc_extracted_data:
    print("User 1 Persona:", data['user1_persona'])
    print("User 2 Persona:", data['user2_persona'])
    print("Context:", data['context'])
    print("Prompt:", data['prompt'])
    print("Expected response:", data['prompt_response'])
    print()

FileNotFoundError: [Errno 2] No such file or directory: 'Synthetic-Persona-Chat_valid.csv'

In [ ]:
# baseline validation
def process_validation_data(csv_filename):
    extracted_data = extract_conversation_parts(csv_filename)
    trunc_extracted_data = extracted_data[:15]  # Taking only the first 15 rows
    
    model_responses = []
    
    for data in trunc_extracted_data:
        context = f"User 1: {data['user1_persona']}\n\n User 2: {data['user2_persona']}\n \
            First 6 pieces of conversation: {data['context']}"
        prompt = f"{data['prompt']}"
        generated_response = generate_persona_prompt(prompt, context)
        model_responses.append(generated_response)
    
    return model_responses

csv_filename = r'Synthetic-Persona-Chat_valid.csv'

model_responses = process_validation_data(csv_filename)

for response in model_responses:
    print("Generated Response:", response)
    print()


Generated Response: User 2: Besides moving objects, I enjoy meditating to calm my mind.
User 2: I also like watching documentaries about science and space.
User 1: That's really interesting! Do you have a favorite documentary?
User 2: Yes, I love anything that explores the mysteries of the universe.
User 1: Sounds like you have a curious mind! What else fascinates you about space?

Generated Response: User 2: Yeah, it's opened my eyes to a lot of things. I never gave much thought to animal rights before.

Generated Response: User 2: Yeah, it's a classic. Have you seen any other good war documentaries recently?

Generated Response: User 2: Investment banking is lucrative but intense. Have you thought about other career paths in finance? It's always good to explore your options before committing. What draws you to investment banking specifically?

Generated Response: User 2: Definitely, live music is always better than listening to recordings. The energy is incredible at concerts. Plus, 

In [ ]:
def process_validation_data_finetuned(csv_filename):
    extracted_data = extract_conversation_parts(csv_filename)
    trunc_extracted_data = extracted_data[:15]  # Taking only the first 15 rows
    
    model_responses = []
    
    for data in trunc_extracted_data:
        context = f"User 1: {data['user1_persona']}\n\n User 2: {data['user2_persona']}\n \
            First 6 pieces of conversation: {data['context']}"
        prompt = f"{data['prompt']}"
        generated_response = generate_persona_prompt_finetuned(prompt, context)
        model_responses.append(generated_response)
    
    return model_responses

csv_filename = r'Synthetic-Persona-Chat_valid.csv'

model_responses_finetuned = process_validation_data_finetuned(csv_filename)

for response in model_responses_finetuned:
    print("Generated Response:", response)
    print()

Generated Response: User 2: I also had to have a transplant when I was younger.

Generated Response: User 2: Me too! I love animals so much. I wish I could have a pet, but I can't have one in here.

Generated Response: User 2: Yes, it's very interesting.

Generated Response: User 2: That's a good field to go into. It can be very lucrative.

Generated Response: User 2: Me too. I've heard they're even better in person.

Generated Response: User 2: I bet he is. Dogs are the best companions.

Generated Response: User 2: My name is Sarah.

Generated Response: User 2: Do you have a favorite kind of horse to ride?

Generated Response: User 2: That sounds really exciting! Do you ever get scared?

Generated Response: User 2: Oh, I love New Kids on the Block! My favorite song is "Step By Step."

Generated Response: User 2: Roses are beautiful! I also like to grow sunflowers and daisies.

Generated Response: User 2: You definitely should. I think you'll really like it.

Generated Response: User 2

In [ ]:
single_expected_responses = [data['prompt_response'] for data in trunc_extracted_data]
full_expected_responses = [data['finished_convo'] for data in trunc_extracted_data]

single_model_responses = [response.split('\n')[0] for response in model_responses]
single_model_responses

['User 2: Besides moving objects, I enjoy meditating to calm my mind.',
 "User 2: Yeah, it's opened my eyes to a lot of things. I never gave much thought to animal rights before.",
 "User 2: Yeah, it's a classic. Have you seen any other good war documentaries recently?",
 "User 2: Investment banking is lucrative but intense. Have you thought about other career paths in finance? It's always good to explore your options before committing. What draws you to investment banking specifically?",
 "User 2: Definitely, live music is always better than listening to recordings. The energy is incredible at concerts. Plus, it's a great way to unwind after a long week at work. I can't wait to get there and just enjoy the show without any distractions.",
 "User 2: It's amazing how dogs can become such an important part of our lives, isn't it? They're so loyal and loving. Barnaby must bring a lot of joy into your life.",
 "User 2: I'm Sarah. Nice to meet you! By the way, do you have any favorite skate

In [ ]:
single_model_responses_finetuned = [response.split('\n')[0] for response in model_responses_finetuned]
single_model_responses_finetuned

['User 2: I also had to have a transplant when I was younger.',
 "User 2: Me too! I love animals so much. I wish I could have a pet, but I can't have one in here.",
 "User 2: Yes, it's very interesting.",
 "User 2: That's a good field to go into. It can be very lucrative.",
 "User 2: Me too. I've heard they're even better in person.",
 'User 2: I bet he is. Dogs are the best companions.',
 'User 2: My name is Sarah.',
 'User 2: Do you have a favorite kind of horse to ride?',
 'User 2: That sounds really exciting! Do you ever get scared?',
 'User 2: Oh, I love New Kids on the Block! My favorite song is "Step By Step."',
 'User 2: Roses are beautiful! I also like to grow sunflowers and daisies.',
 "User 2: You definitely should. I think you'll really like it.",
 "User 2: It's a great way to stay active and clear your mind.",
 'User 2: My favorite Britney Spears song is Toxic.',
 "User 2: Oh, I've always wanted to visit Canada."]

In [ ]:
bertscore_result = calculate_bertscore(single_expected_responses, single_model_responses)
print(f"BERTScore: {bertscore_result}")

BERTScore: {'precision': 0.6617485880851746, 'recall': 0.5241777300834656, 'f1': 0.5815606713294983}


In [ ]:
bertscore_result_finetuned = calculate_bertscore(single_expected_responses, single_model_responses_finetuned)
print(f"BERTScore: {bertscore_result_finetuned}")

BERTScore: {'precision': 0.6779415607452393, 'recall': 0.6920197010040283, 'f1': 0.6822169423103333}
